- This notebook
  - calls JQUANTS API to get the list of the stocks listed in the Tokyo Stock Exchange.
  - calls Yahoo Finance API to get the latest stock related information such as Price, PE ratio, market cap etc
  - create a CSV file

In [5]:
#import libraries
import requests
import json
import pandas as pd
import yfinance as yf
import datetime as datetime
from tqdm import tqdm
import pandas as pd
from sqlalchemy import create_engine, Column, String, Integer, MetaData, Table
from sqlalchemy.dialects.postgresql import NUMERIC, BIGINT
import psycopg2

# load const
with open('parameters.json', 'r') as file:
    data = json.load(file)

# Define the database connection string (update with your PostgreSQL details)
DATABASE_URI = data["DATABASE_URI"]
mail_address = data["mailaddress"]
password = data["password"]

# Create an engine and metadata object
engine = create_engine(DATABASE_URI)
metadata = MetaData()

In [6]:
data={"mailaddress":mail_address, "password":password}
r_post = requests.post("https://api.jquants.com/v1/token/auth_user", data=json.dumps(data))
rToken = r_post.json()["refreshToken"]
REFRESH_TOKEN = rToken
r_post = requests.post(f"https://api.jquants.com/v1/token/auth_refresh?refreshtoken={REFRESH_TOKEN}")
idToken = r_post.json()["idToken"]

In [7]:
# 上場銘柄一覧
headers = {'Authorization': 'Bearer {}'.format(idToken)}
r = requests.get("https://api.jquants.com/v1/listed/info", headers=headers)

# Pandas DafaFrameへ変換
r_dict = r.json()
df = pd.DataFrame(r_dict['info'])

In [8]:
df["MarketCodeCleansed"] = df[df["MarketCode"].isin(["0111", "0112"])]["Code"].str[:4] + ".T"

In [9]:
ticker = []
ticker = df[df["MarketCode"].isin(["0111", "0112"])]["MarketCodeCleansed"].values.tolist()

In [10]:
df_master_yf = pd.DataFrame(columns=["Code", "previousClose", "trailingPE", "volume", "marketCap", "fiftyTwoWeekLow", "fiftyTwoWeekHigh", "revenuePerShare"])
columns_yf = ["Code", "previousClose", "trailingPE", "volume", "marketCap", "fiftyTwoWeekLow", "fiftyTwoWeekHigh", "revenuePerShare"]
listInfo = ["previousClose", "trailingPE", "volume", "marketCap", "fiftyTwoWeekLow", "fiftyTwoWeekHigh", "revenuePerShare"]

for tk in tqdm(ticker): 
    stock = yf.Ticker(tk)
    data = []
    data.append(tk)
    for info in listInfo:
        try:
            data.append(stock.info[info])
        except:
            data.append("0")     
            
    df_yf_temp = pd.DataFrame(data=[data], columns=columns_yf)
    df_master_yf = pd.concat([df_master_yf, df_yf_temp], ignore_index=True)
#df_yf.head()

  0%|                                                                                                                                                    | 0/3252 [00:00<?, ?it/s]/tmp/ipykernel_635052/2638317538.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_master_yf = pd.concat([df_master_yf, df_yf_temp], ignore_index=True)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3252/3252 [07:06<00:00,  7.63it/s]


In [11]:
df_master_yf["MarketCodeCleansed"] = df_master_yf["Code"]
df_master = pd.merge(df, df_master_yf, on='MarketCodeCleansed')
df_master = df_master.drop('Code_y', axis=1)
df_master.dropna(subset=["trailingPE", "marketCap"], inplace=True)
df_master["trailingPE"] = pd.to_numeric(df_master["trailingPE"], errors='coerce')
df_master["marketCap"] = pd.to_numeric(df_master["marketCap"], errors='coerce')
df_master.rename(columns={'Code_x': 'ShokenCode'}, inplace=True)

In [26]:
from datetime import datetime, timedelta
yesterday = pd.Timestamp(datetime.today().date() - timedelta(days=1))

In [27]:
df_master["Date"] = yesterday

In [28]:
df_master.head()

,Date,ShokenCode,CompanyName,CompanyNameEnglish,Sector17Code,Sector17CodeName,Sector33Code,Sector33CodeName,ScaleCategory,MarketCode,MarketCodeName,MarketCodeCleansed,previousClose,trailingPE,volume,marketCap,fiftyTwoWeekLow,fiftyTwoWeekHigh,revenuePerShare
0,2024-09-16,13010,極洋,"KYOKUYO CO.,LTD.",1,食品,0050,水産・農林業,TOPIX Small 2,0111,プライム,1301.T,4180.0,7.443806,71200,51370188800,3400.0,4345.0,24070.232
1,2024-09-16,13320,ニッスイ,Nissui Corporation,1,食品,0050,水産・農林業,TOPIX Mid400,0111,プライム,1332.T,910.3,12.378459,1163800,286492033024,680.5,1002.5,2740.213
2,2024-09-16,13330,マルハニチロ,Maruha Nichiro Corporation,1,食品,0050,水産・農林業,TOPIX Mid400,0111,プライム,1333.T,3033.0,7.502794,360000,155523416064,2421.0,3408.0,20570.352
3,2024-09-16,13750,雪国まいたけ,"YUKIGUNI MAITAKE CO.,LTD.",1,食品,0050,水産・農林業,TOPIX Small 2,0111,プライム,1375.T,999.0,29.674200,45200,40332832768,869.0,1173.0,859.602
4,2024-09-16,13760,カネコ種苗,"KANEKO SEEDS CO.,LTD.",1,食品,0050,水産・農林業,TOPIX Small 2,0112,スタンダード,1376.T,1359.0,13.274593,11100,15452952576,1340.0,1546.0,5402.386


In [31]:
#insert data into PostgresDB
df_master.to_sql('t_daily_stock_perf', con=engine, if_exists='append', index=False)

NotSupportedError: (psycopg2.errors.FeatureNotSupported) cannot convert infinity to numeric

[SQL: INSERT INTO t_daily_stock_perf ("Date", "ShokenCode", "CompanyName", "CompanyNameEnglish", "Sector17Code", "Sector17CodeName", "Sector33Code", "Sector33CodeName", "ScaleCategory", "MarketCode", "MarketCodeName", "MarketCodeCleansed", "previousClose", ... 446909 characters truncated ... , %(marketCap__999)s, %(fiftyTwoWeekLow__999)s, %(fiftyTwoWeekHigh__999)s, %(revenuePerShare__999)s)]
[parameters: {'Sector17Code__0': '9', 'MarketCodeName__0': 'プライム', 'ShokenCode__0': '68440', 'CompanyName__0': '新電元工業', 'volume__0': 41500, 'revenuePerShare__0': 9964.845, 'Sector33Code__0': '3650', 'trailingPE__0': 0.0, 'previousClose__0': 2295.0, 'CompanyNameEnglish__0': 'Shindengen Electric Manufacturing Co.,Ltd.', 'fiftyTwoWeekHigh__0': 3195.0, 'fiftyTwoWeekLow__0': 2259.0, 'ScaleCategory__0': 'TOPIX Small 2', 'marketCap__0': 23894757376, 'Date__0': datetime.datetime(2024, 9, 16, 0, 0), 'MarketCode__0':

In [1]:
import yfinance as yf

In [3]:
yf.Ticker('1780.T').info

{'address1': 'Kitamachi 22-1',
 'city': 'Komagane',
 'zip': '399-4195',
 'country': 'Japan',
 'phone': '81 26 581 6010',
 'fax': '81 26 582 3966',
 'website': 'https://www.yamaura.co.jp',
 'industry': 'Engineering & Construction',
 'sector': 'Industrials',
 'longBusinessSummary': 'Yamaura Corporation operates as a construction company in Japan. The company operates in three segments: Construction, Engineering, and Development. The Construction segment provides construction and civil engineering work. The Engineering segment engages in design and construction of bridges, telecommunications, penstock construction, etc. The development segment offers real estate services such as in-house development, as well as buying, selling, leasing, and mediating. Yamaura Corporation was founded in 1920 and is headquartered in Komagane, Japan.',
 'fullTimeEmployees': 416,
 'companyOfficers': [{'maxAge': 1,
   'name': 'Mr. Masaki  Yamaura',
   'age': 52,
   'title': 'President & Representative Director

In [ ]:
df_master.replace([np.inf, -np.inf], np.nan, inplace=True)